### Imports

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
%run "./instance.ipynb"

### Optimization Problems

In [134]:
def solve_master(potential_hubs, capacity_levels, new_hub_capacities, fixed_set_up_costs_with_capacity_level, commodities):


    # suppress Gurobi output
    environment = gp.Env(empty = True)
    environment.setParam("OutputFlag", 0)
    environment.start()

    # model
    master = gp.Model('master', env = environment)

    # variables + objective function
    z = master.addVars(potential_hubs, capacity_levels, obj = fixed_set_up_costs_with_capacity_level, vtype = GRB.BINARY, name = "z")
    eta = master.addVar(1, obj = 1, name = 'eta')

    # model sense
    master.ModelSense = GRB.MINIMIZE

    # constraints
    master.addConstr(gp.quicksum(gp.quicksum(new_hub_capacities[i, q] * z[i, q] for i in potential_hubs) for q in capacity_levels) >= gp.quicksum(demand_flattened[k - 1] for k in commodities))
    master.addConstrs(gp.quicksum(z[i, q] for q in capacity_levels) <= 1 for i in potential_hubs)

    # update and solve the model
    master.update()
    master.optimize()

    z_hat = {}

    for i in potential_hubs:
        for q in capacity_levels:
            z_hat[(i, q)] = z[i, q].X

    master_opt_val = master.ObjVal

    return z_hat, master_opt_val

In [136]:
z_hat = solve_master(potential_hubs, capacity_levels, new_hub_capacities, fixed_set_up_costs_with_capacity_level, commodities)[0]
master_opt_val = solve_master(potential_hubs, capacity_levels, new_hub_capacities, fixed_set_up_costs_with_capacity_level, commodities)[1]

In [208]:
def solve_dsp(z_hat, potential_hubs, new_hub_capacities, commodities):


    # suppress Gurobi output
    environment = gp.Env(empty = True)
    environment.setParam("OutputFlag", 0)
    environment.start()

    # model
    dsp = gp.Model('dsp', env = environment)

    # variables + objective function
    alpha = dsp.addVars(commodities, name = "alpha")
    u = dsp.addVars(potential_hubs, commodities, lb = 0, name = "u")
    v = dsp.addVars(potential_hubs, lb = 0, name = "v")

    obj = gp.quicksum(alpha[k] for k in commodities) - gp.quicksum(z_hat[i, q] * u[i, k] for q in capacity_levels for i in potential_hubs for k in commodities) - gp.quicksum(new_hub_capacities[i, q] * z_hat[i, q] * v[i] for q in capacity_levels for i in potential_hubs)

    dsp.setObjective(obj)

    # model sense
    dsp.ModelSense = GRB.MAXIMIZE

    # constraints
    for i in potential_hubs:
        for j in potential_hubs:
            for k in commodities:
                dsp.addConstr(alpha[k] - u[i, k] - u[j, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))
                dsp.addConstr(alpha[k] - u[i, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))

    # update and solve the model
    dsp.update()
    dsp.write("solve_dsp.lp")
    dsp.optimize()

    # optimal solution
    for var in dsp.getVars():
        print(f"{var.VarName} = {var.x}")

    return None # dsp_opt_soln, dsp_opt_val

In [209]:
solve_dsp(z_hat, potential_hubs, new_hub_capacities, commodities)

GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

### Algorithm

In [7]:
def benders_decomp():
    
    upper_bound = np.inf
    t = 0
    dsp_extreme_pts = []
    terminate = False

    while terminate == False:
        master_opt_soln, master_opt_val = solver_master() 

        if master_opt_val == upper_bound:
            terminate = True
        else:
            dsp_opt_soln, dsp_opt_val = solve_dual_subproblem()
            dsp_extreme_pts.append(dsp_opt_soln)

            if dsp_opt_val + ... < upper_bound:
                upper_bound = dsp_opt_val + ...

        t += 1

    return None # benders_opt_soln